In [1]:
import numpy as np
import cv2
import os, sys
import pandas as pd
import shutil
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm

2024-06-16 19:41:28.807618: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-16 19:41:28.807739: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-16 19:41:29.049566: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def load_frames(mode = "train"):
    """Extracts frames from the video
    NOTE: The function checks if directory exists or not,
    if the directory already exists, it does not extract 
    the frames. Be sure that there is no already existing directory
    and function let the function entirely."""
    dir_path = os.path.join("./data", mode+'_imgs')
    if not os.path.isdir(dir_path):
        os.makedirs(dir_path)
        vidcap = cv2.VideoCapture('./data/{}.mp4'.format(mode))
        success,image = vidcap.read()
        count = 0
        while success:
            cv2.imwrite("%s/%d.jpg" % (dir_path,count), image)     # save frame as JPEG file      
            success,image = vidcap.read()
            count += 1
            if count % 1000 == 0:
                print("%d frames read" % count)
    else:
        print("Video frames already extracted \nSkipping extraction...")

    frame_cnt = len(os.listdir(dir_path))
    vid_frames = np.empty((frame_cnt,480,640,3),dtype='uint8')
    for i in range(0,frame_cnt):
                frame = cv2.imread(dir_path + '/' + str(i) + ".jpg")
                # print(",",frame.shape)
                vid_frames[i] = frame
                sys.stdout.write("\rLoading frame " + str(i))
    return vid_frames
load_frames()

array([], shape=(0, 480, 640, 3), dtype=uint8)

In [3]:
# class OptFlowNet:

#     def __init__(self, WINSIZE, WEIGHTS, EPOCHS, BATCH_SIZE):
#         self.WINSIZE = WINSIZE
#         self.WEIGHTS = WEIGHTS
#         self.EPOCHS = EPOCHS
#         self.BATCH_SIZE = BATCH_SIZE
#         self.optflow_dir = "optical_flow_data"  # Directory to save optical flow images

#     def process_frame(self, frame):
#         frame = cv2.resize(frame, (self.WINSIZE[1], self.WINSIZE[0]), interpolation=cv2.INTER_AREA)
#         frame = frame / 127.5 - 1.0  # standardize the image to mean = 0 and std = 1
#         return frame

#     def optflow(self, frame1, frame2):
#         frame1 = frame1[200:400]
#         frame1 = cv2.resize(frame1, (0, 0), fx=0.4, fy=0.5)
#         frame2 = frame2[200:400]
#         frame2 = cv2.resize(frame2, (0, 0), fx=0.4, fy=0.5)
#         flow = np.zeros_like(frame1)
#         prev = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
#         nxt = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
#         flow_data = cv2.calcOpticalFlowFarneback(prev, nxt, None, 0.4, 1, 12, 2, 8, 1.2, 0)
#         mag, ang = cv2.cartToPolar(flow_data[..., 0], flow_data[..., 1])
#         flow[..., 1] = 255  # saturation
#         flow[..., 0] = ang * 180 / np.pi / 2  # hue
#         flow[..., 2] = (mag * 15).astype(int)  # value
#         return flow

#     def prep_data(self, video_file, label_file, shuffle=False, wipe=False, mode="train"):
#         print("Decoding Labels...")
#         if mode != "test":
#             speed_data = np.array(pd.read_csv(label_file, header=None))
#             speed_data = speed_data.flatten()[:-1]  # assuming the last entry is redundant
#             print(f"Loaded {len(speed_data)} labels")
#         else:
#             shuffle = False
#             speed_data = np.array([])

#         if wipe and os.path.isdir(self.optflow_dir):
#             print("Wiping preprocessed data...")
#             shutil.rmtree(self.optflow_dir)

#         processed_video = None
#         if not os.path.isdir(self.optflow_dir):
#             print("Preprocessing video...")
#             os.mkdir(self.optflow_dir)
#             vid = cv2.VideoCapture(video_file)
#             frame_cnt = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
#             processed_video = np.empty((frame_cnt - 1, self.WINSIZE[0], self.WINSIZE[1], 3), dtype='uint8')
#             success, prev = vid.read()
#             i = 0
#             while True:
#                 success, nxt = vid.read()
#                 if not success:
#                     break
#                 flow = self.optflow(prev, nxt)
#                 prev = nxt
#                 flow = cv2.resize(flow, self.WINSIZE, interpolation=cv2.INTER_AREA)
#                 processed_video[i] = flow / 127.5 - 1.0
#                 cv2.imwrite(f"{self.optflow_dir}/{i}.png", flow)
#                 sys.stdout.write(f"\rProcessed {i} frames")
#                 i += 1
#             print(f"\nDone processing {frame_cnt} frames")
#         else:
#             print("Found preprocessed data")
#             frame_cnt = len(os.listdir(self.optflow_dir))
#             processed_video = np.empty((frame_cnt, self.WINSIZE[0], self.WINSIZE[1], 3), dtype='float32')
#             for i in range(frame_cnt):
#                 flow = cv2.imread(f"{self.optflow_dir}/{i}.png")
#                 flow = self.process_frame(flow)
#                 processed_video[i] = flow
#                 sys.stdout.write(f"\rLoading frame {i}")
#             print(f"\nDone loading {frame_cnt} frames")

#         if shuffle:
#             print("Shuffling data")
#             randomize = np.arange(len(processed_video))
#             np.random.shuffle(randomize)
#             processed_video = processed_video[randomize]
#             speed_data = speed_data[randomize]

#         print("Done prepping data")
#         return processed_video, speed_data

#     def create_model(self):
#         print("Compiling Model...")
#         self.model = Sequential()
#         self.model.add(Conv2D(32, (8, 8), padding='same', strides=(4, 4), input_shape=(self.WINSIZE[0], self.WINSIZE[1], 2)))
#         self.model.add(Activation('relu'))
#         self.model.add(Conv2D(64, (8, 8), padding='same', strides=(4, 4)))
#         self.model.add(Activation('relu'))
#         self.model.add(Conv2D(128, (4, 4), padding='same', strides=(2, 2)))
#         self.model.add(Activation('relu'))
#         self.model.add(Conv2D(128, (2, 2), padding='same', strides=(1, 1)))
#         self.model.add(Activation('relu'))
#         self.model.add(Flatten())
#         self.model.add(Dropout(0.5))
#         self.model.add(Dense(128))
#         self.model.add(Activation('relu'))
#         self.model.add(Dropout(0.5))
#         self.model.add(Dense(128))
#         self.model.add(Dropout(0.5))
#         self.model.add(Dense(1))
#         self.model.compile(optimizer='adam', loss='mse')

#     def load_weights(self):
#         try:
#             print("Loading weights...")
#             self.model.load_weights(self.WEIGHTS)
#             return True
#         except ValueError:
#             print("Unable to load weights. Model has changed.")
#             print("Please retrain the model.")
#             return False
#         except IOError:
#             print("Unable to load weights. No previous weights found.")
#             print("Please train the model.")
#             return False

#     def train(self, X_src, Y_src, val_split, wipe, n_epochs=50, batch_size=32):
#         X, Y = self.prep_data(X_src, Y_src, shuffle=True, wipe=wipe)
#         print(f"TRAIN: Images - {X.shape}, Labels - {Y.shape}")
#         X = X[:, :, :, [0, 2]]  # Extract hue and value channels in data

#         print("Training...")
#         self.model.fit(X, Y, batch_size=batch_size, epochs=n_epochs, validation_split=val_split)
#         print("Done training. Saving weights...")
#         self.model.save_weights(self.WEIGHTS)

#     def evaluate(self, X_src, Y_src):
#         X_eval, Y_eval = self.prep_data(X_src, Y_src, shuffle=False)
#         print(f"EVAL: Images - {X_eval.shape}, Labels - {Y_eval.shape}")

#         success = self.load_weights()
#         if success:
#             print("Evaluating...")
#             print(self.model.evaluate(X_eval, Y_eval))
#             print("Done evaluating")
#         else:
#             print("Evaluation failed with improper weights.")

#     def play(self, X_src, Y_src, output_video_path=None):
#         print("Reading Inputs...")
#         F = F = load_frames("train")  
#         X, Y = self.prep_data(X_src, Y_src, shuffle=False)

#         if output_video_path:
#             rec = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'MJPG'), 40, (640, 480), True)

#         success = self.load_weights()
#         if success:
#             print("Playing video...")
#             for f, x, y in tqdm(zip(F, X, Y)):
#                 frame = f
#                 pred_y = self.model.predict(np.array([x[:, :, [0, 2]]]))[0, 0]
#                 error = abs(y - pred_y)
#                 font = cv2.FONT_HERSHEY_SIMPLEX
#                 cv2.putText(frame, f'True: {y}', (30, 50), font, 1, (0, 255, 255), 2)
#                 cv2.putText(frame, f'Pred: {pred_y}', (30, 150), font, 1, (0, 255, 255), 2)
#                 cv2.putText(frame, f'Error: {error}', (30, 250), font, 1, (0, 255, 255), 2)
#                 if output_video_path:
#                     rec.write(frame)
#             print("Done playing")
#             if output_video_path:
#                 rec.release()
#         else:
#             print("Failed to load proper weights.")

In [4]:

class OptFlowNet:

    def __init__(self, WINSIZE, WEIGHTS, EPOCHS, BATCH_SIZE):
        self.WINSIZE = WINSIZE
        self.WEIGHTS = WEIGHTS
        self.EPOCHS = EPOCHS
        self.BATCH_SIZE = BATCH_SIZE
        self.optflow_dir = "optical_flow_data"  # Directory to save optical flow images

    def process_frame(self, frame):
        frame = cv2.resize(frame, (self.WINSIZE[1], self.WINSIZE[0]), interpolation=cv2.INTER_AREA)
        frame = frame / 127.5 - 1.0  # standardize the image to mean = 0 and std = 1
        return frame

    def optflow(self, frame1, frame2):
        frame1 = frame1[200:400]
        frame1 = cv2.resize(frame1, (0, 0), fx=0.4, fy=0.5)
        frame2 = frame2[200:400]
        frame2 = cv2.resize(frame2, (0, 0), fx=0.4, fy=0.5)
        flow = np.zeros_like(frame1)
        prev = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        nxt = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        flow_data = cv2.calcOpticalFlowFarneback(prev, nxt, None, 0.4, 1, 12, 2, 8, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow_data[..., 0], flow_data[..., 1])
        flow[..., 1] = 255  # saturation
        flow[..., 0] = ang * 180 / np.pi / 2  # hue
        flow[..., 2] = (mag * 15).astype(int)  # value
        return flow

    def prep_data(self, video_file, label_file, shuffle=False, wipe=False, mode="train"):
        print("Decoding Labels...")
        if mode != "test":
            speed_data = np.array(pd.read_csv(label_file, header=None))
            speed_data = speed_data.flatten()[:-1]  # assuming the last entry is redundant
            print(f"Loaded {len(speed_data)} labels")
        else:
            shuffle = False
            speed_data = np.array([])

        if wipe and os.path.isdir(self.optflow_dir):
            print("Wiping preprocessed data...")
            shutil.rmtree(self.optflow_dir)

        processed_video = None
        if not os.path.isdir(self.optflow_dir):
            print("Preprocessing video...")
            os.mkdir(self.optflow_dir)
            vid = cv2.VideoCapture(video_file)
            frame_cnt = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
            processed_video = np.empty((frame_cnt - 1, self.WINSIZE[0], self.WINSIZE[1], 3), dtype='uint8')
            success, prev = vid.read()
            i = 0
            while True:
                success, nxt = vid.read()
                if not success:
                    break
                flow = self.optflow(prev, nxt)
                prev = nxt
                flow = cv2.resize(flow, self.WINSIZE, interpolation=cv2.INTER_AREA)
                processed_video[i] = flow / 127.5 - 1.0
                cv2.imwrite(f"{self.optflow_dir}/{i}.png", flow)
                sys.stdout.write(f"\rProcessed {i} frames")
                i += 1
            print(f"\nDone processing {frame_cnt} frames")
        else:
            print("Found preprocessed data")
            frame_cnt = len(os.listdir(self.optflow_dir))
            processed_video = np.empty((frame_cnt, self.WINSIZE[0], self.WINSIZE[1], 3), dtype='float32')
            for i in range(frame_cnt):
                flow = cv2.imread(f"{self.optflow_dir}/{i}.png")
                flow = self.process_frame(flow)
                processed_video[i] = flow
                sys.stdout.write(f"\rLoading frame {i}")
            print(f"\nDone loading {frame_cnt} frames")

        if shuffle:
            print("Shuffling data")
            randomize = np.arange(len(processed_video))
            np.random.shuffle(randomize)
            processed_video = processed_video[randomize]
            speed_data = speed_data[randomize]

        print("Done prepping data")
        return processed_video, speed_data

    def create_model(self):
        print("Compiling Model...")
        self.model = Sequential()
        self.model.add(Conv2D(32, (8, 8), padding='same', strides=(4, 4), input_shape=(self.WINSIZE[0], self.WINSIZE[1], 2)))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(64, (8, 8), padding='same', strides=(4, 4)))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(128, (4, 4), padding='same', strides=(2, 2)))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(128, (2, 2), padding='same', strides=(1, 1)))
        self.model.add(Activation('relu'))
        self.model.add(Flatten())
        self.model.add(Dropout(0.5))
        self.model.add(Dense(128))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(128))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(1))
        self.model.compile(optimizer='adam', loss='mse')

    def load_frames(self, video_file):
        vid = cv2.VideoCapture(video_file)
        frames = []
        while True:
            success, frame = vid.read()
            if not success:
                break
            frames.append(frame)
        vid.release()
        return frames
    
    def load_weights(self):
        try:
            print("Loading weights...")
            self.model.load_weights(self.WEIGHTS)
            return True
        except ValueError:
            print("Unable to load weights. Model has changed.")
            print("Please retrain the model.")
            return False
        except IOError:
            print("Unable to load weights. No previous weights found.")
            print("Please train the model.")
            return False

    def train(self, X_src, Y_src, val_split, wipe, n_epochs=50, batch_size=32):
        X, Y = self.prep_data(X_src, Y_src, shuffle=True, wipe=wipe)
        print(f"TRAIN: Images - {X.shape}, Labels - {Y.shape}")
        X = X[:, :, :, [0, 2]]  # Extract hue and value channels in data

        print("Training...")
        self.model.fit(X, Y, batch_size=batch_size, epochs=n_epochs, validation_split=val_split, verbose=0)  # Set verbose=0 to suppress output
        print("Done training. Saving weights...")
        self.model.save_weights(self.WEIGHTS)

    def evaluate(self, X_src, Y_src):
        X_eval, Y_eval = self.prep_data(X_src, Y_src, shuffle=False)
        print(f"EVAL: Images - {X_eval.shape}, Labels - {Y_eval.shape}")

        success = self.load_weights()
        if success:
            print("Evaluating...")
            self.model.evaluate(X_eval, Y_eval, verbose=0)  # Set verbose=0 to suppress output
            print("Done evaluating")
        else:
            print("Evaluation failed with improper weights.")

    def play(self, X_src, Y_src, output_video_path=None):
        print("Reading Inputs...")
        F = self.load_frames(X_src)
        X, Y = self.prep_data(X_src, Y_src, shuffle=False)

        if output_video_path:
            fourcc = cv2.VideoWriter_fourcc(*'MJPG')
            frame_height, frame_width = F[0].shape[:2]
            rec = cv2.VideoWriter(output_video_path, fourcc, 40, (frame_width, frame_height), True)

        success = self.load_weights()
        if success:
            print("Playing video...")
            # Disable all output
            original_stdout = sys.stdout
            sys.stdout = open(os.devnull, 'w')

            for f, x, y in zip(F, X, Y):
                frame = f
                pred_y = self.model.predict(np.array([x[:, :, [0, 2]]]))[0, 0]
                error = abs(y - pred_y)
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(frame, f'True: {y}', (30, 50), font, 1, (0, 255, 255), 2)
                cv2.putText(frame, f'Pred: {pred_y}', (30, 150), font, 1, (0, 255, 255), 2)
                cv2.putText(frame, f'Error: {error}', (30, 250), font, 1, (0, 255, 255), 2)
                if output_video_path:
                    rec.write(frame)

            # Revert stdout back to normal
            sys.stdout = original_stdout

            print("Done playing")
            if output_video_path:
                rec.release()
        else:
            print("Failed to load proper weights.")

In [6]:
if __name__ == "__main__":
    WINSIZE = (100, 100)
    WEIGHTS = ".weights.h5"
    EPOCHS = 30
    BATCH_SIZE = 32
    
    video_file = "/kaggle/input/q3-computer-vision/train.mp4"
    label_file = "/kaggle/input/q3-computer-vision/train.txt"
    
    model = OptFlowNet(WINSIZE, WEIGHTS, EPOCHS, BATCH_SIZE)

    # Choose mode
    mode = "train"  # Change this to "evaluate" or "play" as needed
    val_split = 0.3
    resume = True
    wipe = True

    # Create the model
    model.create_model()

    # Load weights if resuming
    if resume:
        model.load_weights()

    # Train or evaluate based on the mode
    if mode == "train":
        model.train(video_file, label_file, val_split, wipe, n_epochs=EPOCHS, batch_size=BATCH_SIZE)
    elif mode == "evaluate":
        model.evaluate(video_file, label_file)
    elif mode == "play":
        model.play(video_file, label_file)


Compiling Model...
Loading weights...
Unable to load weights. No previous weights found.
Please train the model.
Decoding Labels...
Loaded 20399 labels
Wiping preprocessed data...


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Preprocessing video...
Processed 20398 frames
Done processing 20400 frames
Shuffling data
Done prepping data
TRAIN: Images - (20399, 100, 100, 3), Labels - (20399,)
Training...


I0000 00:00:1718539948.114884      86 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1718539948.136956      86 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1718539958.526065      86 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1718539959.901341      86 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1718539961.120657      85 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Done training. Saving weights...


qweaddghg

In [7]:
print('here')

here


In [9]:
model = OptFlowNet(WINSIZE, WEIGHTS, EPOCHS, BATCH_SIZE)
mode = "play" 
val_split = 0.3
resume = True
wipe = True


# Create the model
model.create_model()

if resume:
    model.load_weights()

# Train or evaluate based on the mode
if mode == "train":
    model.train(video_file, label_file, val_split, wipe, n_epochs=EPOCHS, batch_size=BATCH_SIZE)
elif mode == "evaluate":
    model.evaluate(video_file, label_file)
elif mode == "play":
    output_video_path = "/kaggle/working/output_video.avi"
    model.play(video_file, label_file, output_video_path)

Compiling Model...
Loading weights...
Reading Inputs...


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Decoding Labels...
Loaded 10797 labels
Found preprocessed data
Loading frame 20398
Done loading 20399 frames
Done prepping data
Loading weights...
Playing video...


/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


KeyboardInterrupt: 

In [ ]:
print('done')

In [9]:
from IPython.display import FileLink

# Use the correct relative path
file_path = 'output_video.avi'  
FileLink(file_path)


/kaggle/working/output_video.avi

# part 2

In [2]:

class OptFlowNet:

    def __init__(self, WINSIZE, WEIGHTS, EPOCHS, BATCH_SIZE):
        self.WINSIZE = WINSIZE
        self.WEIGHTS = WEIGHTS
        self.EPOCHS = EPOCHS
        self.BATCH_SIZE = BATCH_SIZE
        self.optflow_dir = "optical_flow_data"  # Directory to save optical flow images

    def process_frame(self, frame):
        frame = cv2.resize(frame, (self.WINSIZE[1], self.WINSIZE[0]), interpolation=cv2.INTER_AREA)
        frame = frame / 127.5 - 1.0  # standardize the image to mean = 0 and std = 1
        return frame

    def optflow(self, frame1, frame2):
        frame1 = frame1[200:400]
        frame1 = cv2.resize(frame1, (0, 0), fx=0.4, fy=0.5)
        frame2 = frame2[200:400]
        frame2 = cv2.resize(frame2, (0, 0), fx=0.4, fy=0.5)
        flow = np.zeros_like(frame1)
        prev = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        nxt = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        flow_data = cv2.calcOpticalFlowFarneback(prev, nxt, None, 0.4, 1, 12, 2, 8, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow_data[..., 0], flow_data[..., 1])
        flow[..., 1] = 255  # saturation
        flow[..., 0] = ang * 180 / np.pi / 2  # hue
        flow[..., 2] = (mag * 15).astype(int)  # value
        return flow

    def prep_data(self, video_file, label_file, shuffle=False, wipe=False, mode="train"):
        print("Decoding Labels...")
        if mode != "test":
            speed_data = np.array(pd.read_csv(label_file, header=None))
            speed_data = speed_data.flatten()[:-1]  # assuming the last entry is redundant
            print(f"Loaded {len(speed_data)} labels")
        else:
            shuffle = False
            speed_data = np.array([])

        if wipe and os.path.isdir(self.optflow_dir):
            print("Wiping preprocessed data...")
            shutil.rmtree(self.optflow_dir)

        processed_video = None
        if not os.path.isdir(self.optflow_dir):
            print("Preprocessing video...")
            os.mkdir(self.optflow_dir)
            vid = cv2.VideoCapture(video_file)
            frame_cnt = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
            processed_video = np.empty((frame_cnt - 1, self.WINSIZE[0], self.WINSIZE[1], 3), dtype='uint8')
            success, prev = vid.read()
            i = 0
            while True:
                success, nxt = vid.read()
                if not success:
                    break
                flow = self.optflow(prev, nxt)
                prev = nxt
                flow = cv2.resize(flow, self.WINSIZE, interpolation=cv2.INTER_AREA)
                processed_video[i] = flow / 127.5 - 1.0
                cv2.imwrite(f"{self.optflow_dir}/{i}.png", flow)
                sys.stdout.write(f"\rProcessed {i} frames")
                i += 1
            print(f"\nDone processing {frame_cnt} frames")
        else:
            print("Found preprocessed data")
            frame_cnt = len(os.listdir(self.optflow_dir))
            processed_video = np.empty((frame_cnt, self.WINSIZE[0], self.WINSIZE[1], 3), dtype='float32')
            for i in range(frame_cnt):
                flow = cv2.imread(f"{self.optflow_dir}/{i}.png")
                flow = self.process_frame(flow)
                processed_video[i] = flow
                sys.stdout.write(f"\rLoading frame {i}")
            print(f"\nDone loading {frame_cnt} frames")

        if shuffle:
            print("Shuffling data")
            randomize = np.arange(len(processed_video))
            np.random.shuffle(randomize)
            processed_video = processed_video[randomize]
            speed_data = speed_data[randomize]

        print("Done prepping data")
        return processed_video, speed_data

    def create_model(self):
        print("Compiling Model...")
        self.model = Sequential()
        self.model.add(Conv2D(32, (8, 8), padding='same', strides=(4, 4), input_shape=(self.WINSIZE[0], self.WINSIZE[1], 2)))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(64, (8, 8), padding='same', strides=(4, 4)))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(128, (4, 4), padding='same', strides=(2, 2)))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(128, (2, 2), padding='same', strides=(1, 1)))
        self.model.add(Activation('relu'))
        self.model.add(Flatten())
        self.model.add(Dropout(0.5))
        self.model.add(Dense(128))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(128))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(1))
        self.model.compile(optimizer='adam', loss='mse')

    def load_frames(self, video_file):
        vid = cv2.VideoCapture(video_file)
        frames = []
        while True:
            success, frame = vid.read()
            if not success:
                break
            frames.append(frame)
        vid.release()
        return frames
    
    def load_weights(self):
        try:
            print("Loading weights...")
            self.model.load_weights(self.WEIGHTS)
            return True
        except ValueError:
            print("Unable to load weights. Model has changed.")
            print("Please retrain the model.")
            return False
        except IOError:
            print("Unable to load weights. No previous weights found.")
            print("Please train the model.")
            return False

    def train(self, X_src, Y_src, val_split, wipe, n_epochs=50, batch_size=32):
        X, Y = self.prep_data(X_src, Y_src, shuffle=True, wipe=wipe)
        print(f"TRAIN: Images - {X.shape}, Labels - {Y.shape}")
        X = X[:, :, :, [0, 2]]  # Extract hue and value channels in data

        print("Training...")
        self.model.fit(X, Y, batch_size=batch_size, epochs=n_epochs, validation_split=val_split, verbose=0)  # Set verbose=0 to suppress output
        print("Done training. Saving weights...")
        self.model.save_weights(self.WEIGHTS)

    def evaluate(self, X_src, Y_src, output_video_path=None):
        X_eval, Y_eval = self.prep_data(X_src, Y_src, shuffle=False)
        print(f"EVAL: Images - {X_eval.shape}, Labels - {Y_eval.shape}")

        success = self.load_weights()
        if success:
            print("Evaluating...")
            if output_video_path:
                fourcc = cv2.VideoWriter_fourcc(*'MJPG')
                frame_height, frame_width = X_eval[0].shape[:2]
                rec = cv2.VideoWriter(output_video_path, fourcc, 40, (frame_width, frame_height), True)

            for x, y in zip(X_eval, Y_eval):
                pred_y = self.model.predict(np.array([x[:, :, [0, 2]]]))[0, 0]
                error = abs(y - pred_y)
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(x, f'True: {y}', (30, 50), font, 1, (0, 255, 255), 2)
                cv2.putText(x, f'Pred: {pred_y}', (30, 150), font, 1, (0, 255, 255), 2)
                cv2.putText(x, f'Error: {error}', (30, 250), font, 1, (0, 255, 255), 2)
                if output_video_path:
                    rec.write(x)

            if output_video_path:
                rec.release()
            print("Done evaluating")
        else:
            print("Evaluation failed with improper weights.")

    def play(self, X_src, Y_src, output_video_path=None):
        print("Reading Inputs...")
        F = self.load_frames(X_src)
        X, Y = self.prep_data(X_src, Y_src, shuffle=False)

        if output_video_path:
            fourcc = cv2.VideoWriter_fourcc(*'MJPG')
            frame_height, frame_width = F[0].shape[:2]
            rec = cv2.VideoWriter(output_video_path, fourcc, 40, (frame_width, frame_height), True)

        success = self.load_weights()
        if success:
            print("Playing video...")
            # Disable all output
            original_stdout = sys.stdout
            sys.stdout = open(os.devnull, 'w')

            for f, x, y in zip(F, X, Y):
                frame = f
                pred_y = self.model.predict(np.array([x[:, :, [0, 2]]]))[0, 0]
                error = abs(y - pred_y)
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(frame, f'True: {y}', (30, 50), font, 1, (0, 255, 255), 2)
                cv2.putText(frame, f'Pred: {pred_y}', (30, 150), font, 1, (0, 255, 255), 2)
                cv2.putText(frame, f'Error: {error}', (30, 250), font, 1, (0, 255, 255), 2)
                if output_video_path:
                    rec.write(frame)

            # Revert stdout back to normal
            sys.stdout = original_stdout

            print("Done playing")
            if output_video_path:
                rec.release()
        else:
            print("Failed to load proper weights.")

In [3]:
if __name__ == "__main__":
    WINSIZE = (100, 100)
    WEIGHTS = ".weights.h5"
    EPOCHS = 30
    BATCH_SIZE = 32
    
    video_file = "/kaggle/input/q3-computer-vision/train.mp4"
    label_file = "/kaggle/input/q3-computer-vision/train.txt"
    
    model = OptFlowNet(WINSIZE, WEIGHTS, EPOCHS, BATCH_SIZE)

    # Choose mode
    mode = "train"  # Change this to "evaluate" or "play" as needed
    val_split = 0.3
    resume = True
    wipe = True

    # Create the model
    model.create_model()

    # Load weights if resuming
    if resume:
        model.load_weights()

    # Train or evaluate based on the mode
    if mode == "train":
        model.train(video_file, label_file, val_split, wipe, n_epochs=EPOCHS, batch_size=BATCH_SIZE)
    elif mode == "evaluate":
        model.evaluate(video_file, label_file)
    elif mode == "play":
        model.play(video_file, label_file)


Compiling Model...


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Loading weights...
Unable to load weights. No previous weights found.
Please train the model.
Decoding Labels...
Loaded 20399 labels
Preprocessing video...
Processed 20398 frames
Done processing 20400 frames
Shuffling data
Done prepping data
TRAIN: Images - (20399, 100, 100, 3), Labels - (20399,)
Training...


I0000 00:00:1718567092.813813      87 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1718567092.835721      87 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1718567102.450069      89 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1718567103.807350      88 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1718567105.033489      89 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Done training. Saving weights...


In [4]:
print('1')

1


In [5]:
model = OptFlowNet(WINSIZE, WEIGHTS, EPOCHS, BATCH_SIZE)
mode = "play" 
val_split = 0.3
resume = True
wipe = True

video_file = "/kaggle/input/q3-computer-vision/test.mp4"
label_file = "/kaggle/input/q3-computer-vision/test.txt"


model.create_model()

if resume:
    model.load_weights()

# Train or evaluate based on the mode
if mode == "train":
    model.train(video_file, label_file, val_split, wipe, n_epochs=EPOCHS, batch_size=BATCH_SIZE)
elif mode == "evaluate":
    model.evaluate(video_file, label_file)
elif mode == "play":
    output_video_path = "/kaggle/working/output_video_with_optical_flow.avi"
    model.play(video_file, label_file, output_video_path)

Compiling Model...
Loading weights...
Reading Inputs...


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Decoding Labels...
Loaded 10797 labels
Found preprocessed data
Loading frame 20398
Done loading 20399 frames
Done prepping data
Loading weights...
Playing video...


/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Done playing


In [6]:
from IPython.display import FileLink

# Use the correct relative path
file_path = 'output_video_with_optical_flow.avi'  
FileLink(file_path)


/kaggle/working/output_video_with_optical_flow.avi